## Sentiment Analysis in Financial Markets
### Exploratory Data Analysis

Analyze news articles, financial reports, or social media to gauge market sentiment. This project would involve natural language processing (NLP) techniques to understand how sentiment affects stock prices.

### Stocks to focus on 

- Tesla (Tsla)

- Apple Inc. (AAPL)

- Amazon.com Inc. (AMZN)

- Alphabet Inc. (GOOGL)

- Microsoft Corporation (MSFT)

- Facebook, Inc. (META)

- JPMorgan Chase & Co. (JPM)

- NVIDIA Corporation (NVDA)


### Stock Market History Data

In [1]:
import yfinance as yf
import pandas as pd

c:\Users\Ethan Vaz Falcao\anaconda3\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [36]:
df_stocks=pd.read_csv('data/stocks/df_stocks.csv', sep=",")
df_stocks.drop(index=1, inplace=True)
df_stocks.head(3)

,Ticker,TSLA,TSLA.1,TSLA.2,TSLA.3,TSLA.4,TSLA.5,GOOG,GOOG.1,GOOG.2,...,AMZN.2,AMZN.3,AMZN.4,AMZN.5,AAPL,AAPL.1,AAPL.2,AAPL.3,AAPL.4,AAPL.5
0,Price,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,...,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,Volume
2,2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,15.615220069885254,15.678980827331543,15.547722816467285,...,6.6570000648498535,6.695000171661377,6.695000171661377,151998000,7.622499942779541,7.660714149475098,7.585000038146973,7.643214225769043,6.479000091552734,493729600
3,2010-01-05,NaN,NaN,NaN,NaN,NaN,NaN,15.620948791503906,15.6373872756958,15.480475425720215,...,6.5904998779296875,6.734499931335449,6.734499931335449,177038000,7.664286136627197,7.699643135070801,7.6160712242126465,7.656428813934326,6.490199089050293,601904800


In [29]:
df_stocks.isnull().sum()

Ticker      0
AMZN        0
AMZN.1      0
AMZN.2      0
AMZN.3      0
AMZN.4      0
AMZN.5      0
GOOG        0
GOOG.1      0
GOOG.2      0
GOOG.3      0
GOOG.4      0
GOOG.5      0
TSLA      122
TSLA.1    122
TSLA.2    122
TSLA.3    122
TSLA.4    122
TSLA.5    122
MSFT        0
MSFT.1      0
MSFT.2      0
MSFT.3      0
MSFT.4      0
MSFT.5      0
AAPL        0
AAPL.1      0
AAPL.2      0
AAPL.3      0
AAPL.4      0
AAPL.5      0
dtype: int64

In [3]:
import pandas as pd



# Convert Date to datetime type and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Calculate Daily Price Change and Percentage Change for each stock
for stock in ['MSFT', 'AMZN']:  # Add other stocks as needed
    data[f'{stock}_Price_Change'] = data[f'{stock}_Close'].diff()
    data[f'{stock}_Pct_Change'] = data[f'{stock}_Close'].pct_change() * 100

# Calculate Historical Volatility (e.g., 21-day rolling standard deviation of daily returns)
window = 21  # Example for a 21-day rolling window
for stock in ['MSFT', 'AMZN']:
    data[f'{stock}_Volatility'] = data[f'{stock}_Pct_Change'].rolling(window).std()

# Calculate Average Trading Volume over a period (e.g., 30 days)
window = 30  # Example for a 30-day rolling window
for stock in ['MSFT', 'AMZN']:
    data[f'{stock}_Avg_Volume'] = data[f'{stock}_Volume'].rolling(window).mean()

# Display the DataFrame with the new calculations
data


KeyError: 'Date'